# Installation
CliMETAR users some specific packages, that are not available by default. Therefore, you should download and install them before using the other scripts.

## Cartopy
Cartopy is a library for plotting maps. It also handels different projections and handles coordinate systems.

In [1]:
!conda install -qyS cartopy


# All requested packages already installed.



# Data Aquisition

## Natural Earth
For plotting maps, the Natural Earth dataset is nessesary. Please download the nessesary files, and store them in the `CliMETAR/natural_earth` folder:
* [https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip]()
* [https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip]()
* [https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/NE1_LR_LC_SR_W_DR.zip]()

## Station data
Station data is provided in the repository, but should be regularly updated. Use the script below (on a machine with internet access) to regenerate a new file.
Store a new file in the `CliMetar/resources` folder

```{python}
import datetime
import json
import re
import urllib
data = {'generated':datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%SZ'),'networks':{}, 'stations':{}}
with urllib.request.urlopen("https://mesonet.agron.iastate.edu/geojson/networks.geojson") as response:
    networksjson = json.loads(response.read())
    for nw in networksjson['features']:
        if 'ASOS' in nw['id'] or 'AWOS' in nw['id']:
            abbr, name, polygon = nw['id'], nw['properties']['name'], nw['geometry']['coordinates']
            data['networks'][abbr] = {'abbr':abbr,'name':name,'polygon':polygon}
networklist = list(data['networks'].keys())
for i, network in enumerate(networklist):
    with urllib.request.urlopen(f"https://mesonet.agron.iastate.edu/geojson/network/{network}.geojson") as response:
        stationsjson = json.loads(response.read())
        for st in stationsjson['features']:
            abbr, name, elev, tzname = st['properties']['sid'], st['properties']['sname'], st['properties']['elevation'], st['properties']['tzname']
            lat, lon = st['geometry']['coordinates']
            data['stations'][abbr] = {'abbr':abbr,'name':name,'timezone':tzname,'latitude':lat,'longitude':lon,'elevation':elev}      
with open('stations.json','w') as fh:
    json.dump(data,fh)
```